In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import time
import dill
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

# gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
# tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:
def loop_wrapper(func):
    def loop_func(u0,Δx):
        u=u0+0
        for i in range(y.shape[1]-1):
            u=func(u,Δx)
        return u
    return loop_func

In [3]:
def teste_cp(n_cp, Δx):
    f = lambda x: (x**(n_cp+1))*np.exp(0.75*x)
#     f = lambda x: np.exp(-20*x**2)
#     f = lambda x: -np.sin(np.pi*x)-0.5*x**3 + np.where(x > 0, 1, 0)
    x = np.arange(-1-3*Δx, 1+3*Δx, Δx, dtype = float_pres)
    
    return f(x)

def ref_cp(n_cp, Δx):
    df = lambda x: (n_cp+1)*(x**n_cp)*np.exp(0.75*x) + 0.75*(x**(n_cp+1))*np.exp(0.75*x)
#     df = lambda x: -40*x*np.exp(-20*x**2)
#     df = lambda x: -np.pi*np.cos(np.pi*x)-1.5*x**2
    x  = np.arange(-1-3*Δx, 1+3*Δx, Δx, dtype = float_pres)
    
    return df(x)

In [4]:
nomes = [
    "WENO-JS"     , "WENO-Z"      , "WENO-Z+"    , 
    "WENO-JS (M)" , "WENO-Z (M)"  , "WENO-Z+ (M)" , 
    "WENO-JS (MS)", "WENO-Z (MS)" , "WENO-Z+ (MS)", 
    "WENO-JS (BI)", "WENO-Z (BI)" , "WENO-Z+ (BI)",
    "WENO-ZC (M)" , "WENO-ZC (MS)", "WENO-ZC (BI)"
]

API      = API_Numpy
equation = diff_equation

WENOs = {}
WENOs[nomes[ 0]] = simulation(API, equation, WENO_JS, mapping =     null_mapping, map_function = None).DerivadaEspacial
WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z , mapping =     null_mapping, map_function = None).DerivadaEspacial
WENOs[nomes[ 2]] = simulation(API, equation, WENO_Zp, mapping =     null_mapping, map_function = None).DerivadaEspacial
WENOs[nomes[ 3]] = simulation(API, equation, WENO_JS, mapping =     post_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[ 4]] = simulation(API, equation, WENO_Z , mapping =     post_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[ 5]] = simulation(API, equation, WENO_Zp, mapping =     post_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[ 6]] = simulation(API, equation, WENO_JS, mapping =      pre_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[ 7]] = simulation(API, equation, WENO_Z , mapping =      pre_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[ 8]] = simulation(API, equation, WENO_Zp, mapping =      pre_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[ 9]] = simulation(API, equation, WENO_JS, mapping =      pre_mapping, map_function = BI_mapping).DerivadaEspacial
WENOs[nomes[10]] = simulation(API, equation, WENO_Z , mapping =      pre_mapping, map_function = BI_mapping).DerivadaEspacial
WENOs[nomes[11]] = simulation(API, equation, WENO_Zp, mapping =      pre_mapping, map_function = BI_mapping).DerivadaEspacial
WENOs[nomes[12]] = simulation(API, equation, WENO_ZC, mapping = post_inv_mapping, map_function = Henrick_mapping).DerivadaEspacial
WENOs[nomes[13]] = simulation(API, equation, WENO_ZC, mapping =  pre_inv_mapping, map_function = Hong_mapping).DerivadaEspacial
WENOs[nomes[14]] = simulation(API, equation, WENO_ZC, mapping =  pre_inv_mapping, map_function = BI_mapping).DerivadaEspacial

In [5]:
base = 0.5
# Δxs  = 2/20*base**np.arange(0, 10+1)
Δxs  = 2/160*base**np.arange(0, 10+1)
# malha = [2/160, 2/320, 2/640, 2/1280, 2/2560, 2/5120, 2/10240, 2/20480]

fronteira = FronteiraFixa
n_cp = 1

testes    = {}
derivadas = {}
erros     = {}
for nome in nomes:
    
    resultados = []
    derivada   = []
    erro       = []
    for Δx in Δxs:
        u     = teste_cp(n_cp, Δx)
        u_hat = WENOs[nome](u, Δx, fronteira)
        dif   = u_hat - ref_cp(n_cp, Δx)
        
        resultados.append(np.max(np.abs(dif[3:-3])))             # norma L-inf
#         resultados.append(np.sqrt(np.sum(np.square(dif[3:-3])))) # norma L2
#         resultados.append((np.sum(np.abs(dif[3:-3])))            # norma L1
        
        derivada.append(u_hat)
        erro.append(dif[3:-3])
        
    testes[nome]    = resultados
    derivadas[nome] = derivada
    erros[nome]     = erro

In [7]:
derivadas["WENO-Z"][0].dtype

dtype('float64')

In [ ]:
aux_nome = "WENO-Z"

aux_x = Δxs
aux_y = derivadas[aux_nome]

x     = tf.range(-1-3*aux_x[0], 1+3*aux_x[0], aux_x[0], dtype = float_pres)
y     = aux_y[0]
label = tf.repeat([2/Δxs[0]], [len(aux_y[0])])

for i in tf.range(1, len(aux_x)):
    x = tf.concat([x, tf.range(-1-3*aux_x[i], 1+3*aux_x[i], aux_x[i], dtype = float_pres)], axis = 0)
    y = tf.concat([y, aux_y[i]], axis = 0)
    label = tf.concat([label, tf.repeat([2/Δxs[i]], [len(aux_y[i])])], axis = 0)

# x     = tf.range(-1, 1, aux_x[0], dtype = float_pres)
# y     = aux_y[0]
# label = tf.repeat([2/Δxs[0]], [len(aux_y[0])])

# for i in tf.range(1, len(aux_x)):
#     x = tf.concat([x, tf.range(-1, 1, aux_x[i], dtype = float_pres)], axis = 0)
#     y = tf.concat([y, aux_y[i]], axis = 0)
#     label = tf.concat([label, tf.repeat([2/Δxs[i]], [len(aux_y[i])])], axis = 0)

    
df = pd.DataFrame(
    {
        "x"     : x     ,
        "y"     : y     ,
        "label" : label
    }
)

In [ ]:
fig = ply_exp.line(
    data_frame = df    ,
    x          = x     ,
    y          = y     ,
    color      = label
)
fig.add_trace(
    ply_go.Scatter(
        x    = tf.range(-1-3*aux_x[len(Δxs)-1], 1+3*aux_x[len(Δxs)-1], aux_x[len(Δxs)-1], dtype = float_pres),
        y    = ref_cp(n_cp, aux_x[len(Δxs)-1]) , # u_ref, 
        mode = "lines"                                           ,
        line = ply_go.scatter.Line(color="#666666")              ,
        name = "Referência"
    )
)
fig.show()

In [ ]:
fig.write_html("C:/Users/daniw/OneDrive/Documentos/GitHub/Top_mat_A/WENO-BI/ordem_de_convergência_gausiana.html")

In [27]:
testes

{'WENO-JS': [6.294989461190503e-07,
  7.43842375732634e-08,
  9.026347101183385e-09,
  1.1112361486716182e-09,
  1.3783621227898835e-10,
  1.7162495166353486e-11,
  1.1645795439108042e-11,
  1.6867396368525078e-11,
  5.4072302191343624e-11,
  7.177014538228832e-11,
  1.7472334690182834e-10],
 'WENO-Z': [4.06012738360384e-09,
  1.1089708838729617e-10,
  3.210607127379639e-12,
  7.442935157087049e-13,
  2.5135449277513544e-12,
  3.82804898890754e-12,
  9.372058684675721e-12,
  1.4338752407638822e-11,
  4.497735517361434e-11,
  6.853451139932076e-11,
  2.1110313497274547e-10],
 'WENO-Z+': [9.307693498578073e-08,
  1.4010224261978843e-08,
  2.038883230651156e-09,
  2.9058563194572784e-10,
  4.064491972155526e-11,
  5.58764579744464e-12,
  9.372058684675721e-12,
  1.4593659614092758e-11,
  4.497735517361434e-11,
  7.125855461254105e-11,
  2.1110313497274547e-10],
 'WENO-JS (M)': [2.174043590985608e-09,
  6.622529087618734e-11,
  2.041687131859593e-12,
  7.549516567451064e-13,
  2.5135449277

In [28]:
ordem = {}
for nome in nomes:
    l_erro = np.log(testes[nome])/np.log(base)
    l_dif  = l_erro[1:]-l_erro[:-1]
    ordem[nome] = l_dif

In [29]:
ordem

{'WENO-JS': array([ 3.08113512,  3.04278278,  3.02197682,  3.01113857,  3.00562374,
         0.55945013, -0.53442812, -1.68065248, -0.40849406, -1.28361665]),
 'WENO-Z': array([ 5.1942316 ,  5.11023154,  2.10890257, -1.7557799 , -0.60688582,
        -1.2917567 , -0.61348161, -1.64927933, -0.60763183, -1.62304544]),
 'WENO-Z+': array([ 2.73194365,  2.78062899,  2.81074388,  2.83781643,  2.86276256,
        -0.74612542, -0.63890382, -1.62385712, -0.66386439, -1.56681288]),
 'WENO-JS (M)': array([ 5.03685478,  5.01954857,  1.43530563, -1.73526731, -0.60688582,
        -1.60512799, -0.53442812, -1.41496153, -0.78494387, -1.4457334 ]),
 'WENO-Z (M)': array([ 2.19695521, -1.88628032, -0.2333231 , -1.79239407, -0.60688582,
        -1.60512799, -0.33061615, -1.6187735 , -0.57642852, -1.65424875]),
 'WENO-Z+ (M)': array([ 4.33560509,  0.37952485, -0.29044986, -1.36539612, -0.976757  ,
        -1.60512799, -0.59417542, -1.62090518, -0.25959419, -1.9347731 ]),
 'WENO-JS (MS)': array([ 5.04219382,